In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
from pathlib import Path
from pprint import pprint
import re
import sys
from typing import Generator
if '..' not in sys.path: sys.path.append('..')

from openai import AzureOpenAI
from pydantic import BaseModel
from pydantic_yaml import parse_yaml_raw_as, parse_yaml_file_as, to_yaml_str, to_yaml_file



# Generate summarization dataset with LLM
## Setup

In [3]:
AZURE_OPENAI_KEY = os.environ.get('MLLM_AZURE_OPENAI_KEY')
AZURE_OPENAI_ENDPOINT = 'https://hackathon-spanish-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview'
# AZURE_OPENAI_ENDPOINT = 'https://hackathon-spanish-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview'
OPENAI_MODEL_NAME = 'gpt-4o'
# API_VERSION = '2024-05-13'
# API_VERSION = '2024-08-06'
API_VERSION = '2024-08-01-preview'
LANG_MAIN_ID, LANG_MAIN_NAME = 'en', 'English'
LANG_LEARN_ID, LANG_LEARN_NAME = 'es', 'Spanish'

In [4]:
HOME_PATH = Path(os.path.expandvars('$HOME'))
DATA_PATH = HOME_PATH / 'data'
LL_DATA_PATH = DATA_PATH / 'lldata'
LL_CONVS_PATH = LL_DATA_PATH / f'convs_m_{OPENAI_MODEL_NAME}_v_{API_VERSION}'
LL_CONVS_PATH.mkdir(parents=True, exist_ok=True)

## Test OpenAI API

In [6]:
client = AzureOpenAI(
  api_key = AZURE_OPENAI_KEY,
  api_version = API_VERSION,
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
)

In [7]:
response = client.chat.completions.create(
    model=OPENAI_MODEL_NAME, # model = 'deployment_name'.
    messages=[
        {'role': 'system', 'content': 'Assistant is a large language model trained by OpenAI.'},
        {'role': 'user', 'content': 'Who were the founders of Microsoft? Answer without saying names, but you can comment on person\'s appearance and traits.'}
    ]
)

# print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": "chatcmpl-BgGyqQrq7kuRNh8MA7TobBMJzWHtV",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The founders of Microsoft include two individuals. One of them is a bespectacled, tech-savvy individual known for his deep interest in programming and software development from a young age. The other is his school friend, who shares a similar enthusiasm for computing and technology, and is often remembered for his more outgoing and assertive personality. Together, these two visionaries went on to build one of the world's most influential technology companies.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered"